In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
import math

In [2]:
#Reading in 2016 data we cleaned
df2016 = pd.read_csv('data/exported_data.csv')


In [3]:
# Reading in 2020 data we collected 

# Population broken down by age
state13 = pd.read_csv('data/cc-est2019-agesex-13.csv')
state19 = pd.read_csv('data/cc-est2019-agesex-19.csv')
state12 = pd.read_csv('data/cc-est2019-agesex-12.csv')
state27 = pd.read_csv('data/cc-est2019-agesex-27.csv')
state32 = pd.read_csv('data/cc-est2019-agesex-32.csv')
state37 = pd.read_csv('data/cc-est2019-agesex-37.csv')
state39 = pd.read_csv('data/cc-est2019-agesex-39.csv')
state42 = pd.read_csv('data/cc-est2019-agesex-42.csv')
state48 = pd.read_csv('data/cc-est2019-agesex-48.csv')
state55 = pd.read_csv('data/cc-est2019-agesex-55.csv')
state26 = pd.read_csv('data/cc-est2019-agesex-26.csv')
state4 = pd.read_csv('data/cc-est2019-agesex-04.csv')
df_states = state13.append(state19).append(state12).append(state27).append(state32).append(state37).append(state39).append(state42).append(state48).append(state55).append(state26).append(state4)

df_states = df_states[df_states['YEAR'] == 12] #12 represents 2019 (as close to 2020 as we could find)




In [4]:
df_states.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
11,50,13,1,Georgia,Appling County,12,18386,9157,9229,1177,608,569,2359,1186,1173,977,492,485,1520,848,672,14347,7115,7232,13873,6871,7002,6592,3430,3162,4366,2221,2145,4730,2324,2406,3257,1478,1779,1177,608,569,1306,655,651,1324,662,662,1124,598,526,1102,611,491,1127,620,507,1017,491,526,1090,555,535,1132,555,577,1111,537,574,1128,578,550,1241,624,617,1250,585,665,1006,482,524,929,465,464,605,272,333,398,150,248,319,109,210,39.7,37.9,41.2
23,50,13,3,Georgia,Atkinson County,12,8165,4146,4019,573,300,273,1069,550,519,498,259,239,751,373,378,6280,3175,3105,6025,3037,2988,3164,1620,1544,2035,1048,987,2056,1043,1013,1183,573,610,573,300,273,595,307,288,594,303,291,584,300,284,545,272,273,547,280,267,498,269,229,462,228,234,528,271,257,512,265,247,534,297,237,505,241,264,505,240,265,392,192,200,325,161,164,228,105,123,148,71,77,90,44,46,36.6,35.9,37.4
35,50,13,5,Georgia,Bacon County,12,11164,5455,5709,766,381,385,1424,719,705,643,313,330,940,500,440,8651,4204,4447,8331,4042,4289,4219,2125,2094,2788,1387,1401,2725,1340,1385,1878,815,1063,766,381,385,774,390,384,802,404,398,753,381,372,678,357,321,684,348,336,679,346,333,631,290,341,794,403,391,717,372,345,706,350,356,629,313,316,673,305,368,671,318,353,479,228,251,323,131,192,243,90,153,162,48,114,38.6,37.2,39.8
47,50,13,7,Georgia,Baker County,12,3038,1494,1544,175,97,78,281,139,142,135,58,77,227,109,118,2513,1228,1285,2447,1200,1247,947,469,478,618,317,301,883,436,447,719,338,381,175,97,78,138,75,63,176,79,97,162,74,88,167,78,89,169,93,76,156,80,76,151,67,84,142,77,65,196,98,98,190,90,100,251,120,131,246,128,118,238,118,120,178,80,98,131,63,68,69,29,40,103,48,55,47.3,46.8,47.9
59,50,13,9,Georgia,Baldwin County,12,44890,22737,22153,2113,1116,997,4262,2158,2104,1913,991,922,8380,4086,4294,37528,18950,18578,36602,18472,18130,19737,10098,9639,9947,5283,4664,10877,5662,5215,7398,3441,3957,2113,1116,997,2278,1130,1148,2487,1290,1197,4360,2049,2311,5430,2766,2664,2846,1517,1329,2420,1286,1134,2452,1284,1168,2229,1196,1033,2458,1320,1138,2631,1397,1234,2925,1490,1435,2863,1455,1408,2452,1203,1249,2008,916,1092,1333,617,716,863,384,479,742,321,421,36.1,35.9,36.3


In [5]:
# Getting the columns we want
df_states = df_states[['STATE', 'COUNTY', 'STNAME', 'CTYNAME','AGE18PLUS_TOT','AGE18PLUS_MALE','AGE18PLUS_FEM', 'AGE1824_TOT', 'AGE1824_MALE','AGE1824_FEM', 
                       'AGE2544_TOT','AGE2544_MALE','AGE2544_FEM', 'AGE4564_TOT','AGE4564_MALE', 'AGE4564_FEM',
                      'AGE65PLUS_TOT', 'AGE65PLUS_MALE', 'AGE65PLUS_FEM']]

In [6]:
# Dividing by total value to get a percentage 
df_states['1824tot_per'] = df_states['AGE1824_TOT']/df_states['AGE18PLUS_TOT']
df_states['1824M_per'] = df_states['AGE1824_MALE']/df_states['AGE18PLUS_TOT']
df_states['1824F_per'] = df_states['AGE1824_FEM']/df_states['AGE18PLUS_TOT']
df_states['2544tot_per'] = df_states['AGE2544_TOT']/df_states['AGE18PLUS_TOT']
df_states['2544M_per'] = df_states['AGE2544_MALE']/df_states['AGE18PLUS_TOT']
df_states['2544F_per'] = df_states['AGE2544_FEM']/df_states['AGE18PLUS_TOT']
df_states['4564tot_per'] = df_states['AGE4564_TOT']/df_states['AGE18PLUS_TOT']
df_states['4564M_per'] = df_states['AGE4564_MALE']/df_states['AGE18PLUS_TOT']
df_states['4564F_per'] = df_states['AGE4564_FEM']/df_states['AGE18PLUS_TOT']
df_states['65tot_per'] = df_states['AGE65PLUS_TOT']/df_states['AGE18PLUS_TOT']
df_states['65M_per'] = df_states['AGE65PLUS_MALE']/df_states['AGE18PLUS_TOT']
df_states['65F_per'] = df_states['AGE65PLUS_FEM']/df_states['AGE18PLUS_TOT']

In [7]:
df_states.head()

,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per
11,13,1,Georgia,Appling County,13873,6871,7002,1520,848,672,4366,2221,2145,4730,2324,2406,3257,1478,1779,0.109565,0.061126,0.048439,0.314712,0.160095,0.154617,0.340950,0.167520,0.173430,0.234773,0.106538,0.128235
23,13,3,Georgia,Atkinson County,6025,3037,2988,751,373,378,2035,1048,987,2056,1043,1013,1183,573,610,0.124647,0.061909,0.062739,0.337759,0.173942,0.163817,0.341245,0.173112,0.168133,0.196349,0.095104,0.101245
35,13,5,Georgia,Bacon County,8331,4042,4289,940,500,440,2788,1387,1401,2725,1340,1385,1878,815,1063,0.112832,0.060017,0.052815,0.334654,0.166487,0.168167,0.327092,0.160845,0.166247,0.225423,0.097827,0.127596
47,13,7,Georgia,Baker County,2447,1200,1247,227,109,118,618,317,301,883,436,447,719,338,381,0.092767,0.044544,0.048222,0.252554,0.129546,0.123008,0.360850,0.178177,0.182673,0.293829,0.138128,0.155701
59,13,9,Georgia,Baldwin County,36602,18472,18130,8380,4086,4294,9947,5283,4664,10877,5662,5215,7398,3441,3957,0.228949,0.111633,0.117316,0.271761,0.144336,0.127425,0.297170,0.154691,0.142479,0.202120,0.094011,0.108109


In [8]:
df_states = df_states.reset_index()